In [ ]:
import pandas as pd
from utils.plotly_theme import *

In [ ]:
import json

with open("data/input/equipment_data.json", "r") as f:
    data = json.load(f)
    eq_types = [item["eq_type"] for item in data["equipment"]]

equipment_types = list(set(eq_types))
print(equipment_types)

In [ ]:
# test interpolation

from utils.interp import interp_vector

fp = [3.815, 3.955, 4.133, 3.74] # cop
xp = [1_465_000, 1_098_750, 732_500, 366_250] # cap

fp_1 = [3.74, 4.133, 3.955, 3.815] # cop
xp_1 = [366_250, 732_500, 1_098_750, 1_465_000] # cap

x = [1_200_000] # 1_465_200

print(interp_vector(xp, fp, x))
print(interp_vector(xp_1, fp_1, x))

In [ ]:
# Example usage of the decarb-tool

from src.metadata import Metadata
from utils.inputs import get_city_and_zone

# Load location data
location_data = pd.read_csv("data/input/locations.csv")
zip_code = "11229" # Brooklyn, NY -> will be replaced by user input from UI (search by zip-code or city name)
locations = get_city_and_zone(zip_code, location_data)

# Based on zip-code, SET location, climate_zone, time_zone
location = locations.city
ashrae_climate_zone = locations.ashrae_climate_zone
time_zone = locations.time_zone

# Set building type, building vintage, load type and equipment via equipment scenario -> eventually replaced by user input from UI
building_type = "Hospital"
vintage = 2004
load_type = "load_simulated"
equipment_scenarios = ["hr_01"]
emission_scenario = "MidCase"

# create Metadata object to store settings (rest of settings are set by default if not specified)
settings = Metadata.create(
    location=location,
    ashrae_climate_zone=ashrae_climate_zone,
    building_type=building_type,
    vintage=vintage,
    load_type=load_type,
    equipment_scenarios=equipment_scenarios
)
settings.emissions.years = [2025, 2040]
settings.emissions.emission_scenario = emission_scenario
settings.emissions.gea_grid_region = "NYISO"
settings.emissions.time_zone = time_zone

print(settings)

In [ ]:
# Get datasets based on input (location, load_type) saved in settings object
from src.loads import get_load_data
from src.emissions import get_emissions_data

# get load data
load_data = get_load_data(settings)

# get emission data
emissions_data = get_emissions_data(settings)

# inspect load data
load_data.df.head(30)

In [ ]:
# inspect emissions data
emissions_data.df.head(30)

In [ ]:
# Test-load equipment library (not changing the actual sim settings)

from src.equipment import load_library

library = load_library("data/input/equipment_data.json")

hp_a = library.get_equipment("hp01")
print(hp_a)

scenario_a = library.get_scenario("decarb_01")
print(scenario_a)

In [ ]:
from src.energy import loads_to_site_energy

site_energy = loads_to_site_energy(load_data, library, settings.equipment_scenarios, detail=True)

site_energy.head(30)


In [ ]:
from src.energy import site_to_source

source_energy = site_to_source(site_energy, settings=settings.emissions, emissions=emissions_data)

source_energy.head(30)


In [ ]:
#! Update for mutliple scenarios and years

from src.visuals import plot_energy_breakdown

df = source_energy.copy()

plot_energy_breakdown(df)


In [ ]:

from src.visuals import plot_energy_and_emissions

df = source_energy.copy()

year = 2025

plot_energy_and_emissions(df, year)


In [ ]:
#! Update for mutliple scenarios

from src.visuals import plot_emissions_heatmap

df = source_energy.copy()

year = 2040

plot_emissions_heatmap(df, year)


In [ ]:
from src.visuals import plot_meter_timeseries

df = source_energy.copy()

metadata_cols = ['scenario_id']
elec_cols = ['elec_hr_Wh',"elec_awhp_h_Wh","elec_chiller_Wh", 'elec_awhp_c_Wh', 'elec_res_Wh']
gas_cols = ['gas_boiler_Wh']

df = df[metadata_cols + elec_cols + gas_cols]

year = 2025
scenario_id = equipment_scenarios[0]

plot_meter_timeseries(df, year=year, scenario_id=scenario_id, stacked=True, include_gas=True)